#Install libraries

In [ ]:
!pip install transformers
!pip install torch torchvision
!pip install pandas
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load the IMDB dataset:

In [6]:
from google.colab import drive
drive.mount('/content/drive')

# Set the base directory
base_dir ='/content/drive/MyDrive/ENSAE/Cursos/S2/NLP/Proyecto/IMDB/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd

def label(x):
     y = 0
     if x == 'pos':
        y = 1
     return y


df_train = pd.read_excel(base_dir+'train.xlsx') 
df_test = pd.read_excel(base_dir+'test.xlsx') 

df_train['Reviews'] = df_train['Reviews'].astype(str)
df_test['Reviews'] = df_test['Reviews'].astype(str)

df_train['Sentiment'] = df_train['Sentiment'].astype(str)
df_test['Sentiment'] = df_test['Sentiment'].astype(str)

df_train['Sentiment_2'] = df_train['Sentiment'].apply(label)

df_test['Sentiment_2'] = df_test['Sentiment'].apply(label)



X_train = df_train['Reviews'].tolist()
X_test = df_test['Reviews'].tolist()

y_train = df_train['Sentiment_2'].tolist()
y_test = df_test['Sentiment_2'].tolist()


Load the BERT tokenizer

In [8]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

Create PyTorch datasets from the encoded data:

In [9]:
import torch

class IMDBDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings 
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

train_dataset = IMDBDataset(train_encodings, y_train)
test_dataset = IMDBDataset(test_encodings, y_test)


Fine-tune the model on the training data

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Evaluate the model on the test data

In [1]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = np.mean((preds == labels).astype(int))
    return {'accuracy': acc}

trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix='test')


NameError: ignored